# Plotting individual run metrics

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

# Loading files

In [ ]:
if os.name != 'posix':
        path = "\\\\?\\" + os.getcwd()
        prefix = '\\'

else:
    path = os.getcwd()
    prefix = '/'

weights = files_classifier = os.listdir('Weights')

# Plot pre-training loss - for the individual layers
1. load the hyperparameters file to know the number of epochs per layer
2. use this number to slice the pre-training.csv loss file
3. plot each slice - ie each layer

In [ ]:
# Define a dictionary to store hyperparameters
hyperparameters = {}

# Open and read the file
with open("Hyperparameters.txt", "r") as file:
    for line in file:
        if ":" in line:
            # Split each line into key and value pairs
            key, value = line.strip().split(": ")
            if key == "epochs":
                epochs = int(value)

            # Convert the value to the appropriate data type (e.g., int, float, list)
            if key in ["scaleaffine", "scale", "slope", "threshold", "bias"]:
                value = [float(x) for x in value.strip("[]").split(", ")]
            elif key in ["device", "nlayers", "nlayers_proj", "nneurons_proj", "epochs", "epochs_classifier",
                         "batchSize_pretrain", "batchSize_classifier", "test_batchSize"]:
                value = int(value)
            elif key in ["lr", "lr_classifier", "deg", "contrast", "hue"]:
                value = float(value)
            elif key in ["dataset"]:
                value = value  # String values

            # Store the key-value pair in the dictionary
            hyperparameters[key] = value

In [ ]:
# getting the data
df_pretraining = pd.read_csv("pre_training.csv", sep = ',', index_col = 0)
len(df_pretraining)

In [ ]:
# looping over the number of layer and try to extract the loss for that layer (ie chunks of len epochs)
losses = []
for layer in range(hyperparameters["nlayers"]):
    try:
        losses.append(df_pretraining["Pre-training loss"].values[layer*epochs: (layer+1)*epochs])
    except:
        pass
        

In [ ]:
colormap = plt.cm.Reds

colors = [colormap(i) for i in np.linspace(0.5,1,len(losses))]

plt.figure()

for layer in range(len(losses)):
    plt.plot(losses[layer], "-", color = colors[layer], label = "Layer #" + str(layer))

plt.grid()
plt.ylim([0, np.max(df_pretraining["Pre-training loss"].values)])
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Pre-training loss")
plt.show()

# Plot training classifier for the different layers over epochs 

In [ ]:
files_classifier = os.listdir('Classifiers')

In [ ]:
files_classifier

In [ ]:
# Use the filter function to categorize the files
layers_files = [list(filter(lambda x: (int(x.split("#")[1][0]) == layer), files_classifier)) for layer in range(hyperparameters["nlayers"])]

In [ ]:
layers_files = [l for l in layers_files if l != [] ]

In [ ]:
_ = [l.sort(key=lambda x: int(x.split("#")[2][:-4])) for l in layers_files]

In [ ]:
fig, axs = plt.subplots(len(layers_files)+1,2, figsize = [10,10])
train_e, test_e, train_l, test_l = [[] for k in range(len(layers_files))], [[] for k in range(len(layers_files))], [[] for k in range(len(layers_files))], [[] for k in range(len(layers_files))] 

for layer in range(len(layers_files)):
    alphas = np.flip(np.linspace(1,0.2,len(layers_files[layer])))
    try:
        for epoch, file in enumerate(layers_files[layer]):
            df_classifier = pd.read_csv("Classifiers/"+ file, sep = ',', index_col = 0)
            train_err, = axs[layer, 0].plot(df_classifier["Training error"].values, color = "firebrick", alpha = alphas[epoch], label = "Training error")
            train_e[layer].append(df_classifier["Training error"].values[-1])
            test_err, = axs[layer, 0].plot(df_classifier["Testing error"].values, color = "forestgreen", alpha = alphas[epoch], label = "Testing error")
            test_e[layer].append(df_classifier["Testing error"].values[-1])
            axs[layer, 0].set_ylim(0, np.max(df_classifier["Training error"].values))
            
            train_loss, = axs[layer, 1].plot(df_classifier["Training loss"].values, "--", color = "firebrick", alpha = alphas[epoch], label = "Training loss")
            train_l[layer].append(df_classifier["Training loss"].values[-1])
            test_loss, = axs[layer, 1].plot(df_classifier["Testing loss"].values, "--", color = "forestgreen", alpha = alphas[epoch], label = "Testing loss")
            test_l[layer].append(df_classifier["Testing loss"].values[-1])
            axs[layer, 1].set_ylim(0, np.max(df_classifier["Training loss"].values))
        
        axs[layer, 0].grid()
        axs[layer, 0].set_xlabel("Epochs training classifier")
        axs[layer, 0].set_ylabel("Error (%)")
        
        axs[layer, 0].legend([train_err, test_err], ["Training error", "Testing error"])
        
        axs[layer, 1].grid()
        axs[layer, 1].set_xlabel("Epochs training classifier")
        axs[layer, 1].set_ylabel("Loss (CE)")
        axs[layer, 1].legend([train_err, test_err], ["Training loss", "Testing loss"])
            
    except:
        pass
    

alphas = np.flip(np.linspace(1,0.2,len(layers_files)))
for layer in range(len(layers_files)):
    train_err, = axs[-1, 0].plot(np.arange(0,50*len(train_e[layer]), 50), train_e[layer], color = "firebrick", alpha = alphas[layer], label = "Training error")
    test_err, = axs[-1, 0].plot(np.arange(0,50*len(test_e[layer]), 50), test_e[layer], color = "forestgreen", alpha = alphas[layer], label = "Testing error")

    train_loss, = axs[-1, 1].plot(np.arange(0,50*len(train_l[layer]), 50), train_l[layer], "--", color = "firebrick", alpha = alphas[layer], label = "Training loss")
    test_loss, = axs[-1, 1].plot(np.arange(0,50*len(test_l[layer]), 50), test_l[layer], "--", color = "forestgreen", alpha = alphas[layer], label = "Testing loss")

axs[-1, 0].set_ylim(0, np.max(df_classifier["Training error"].values))
axs[-1,0].set_xlabel("Pre-training epochs")
axs[-1,0].set_ylabel("Error (%)")

axs[-1, 1].set_ylim(0, np.max(df_classifier["Training loss"].values))
axs[-1,1].set_xlabel("Pre-training epochs")
axs[-1,1].set_ylabel("Loss (CE)")


axs[-1, 0].grid()
axs[-1, 0].legend([train_err, test_err], ["Training error", "Testing error"])
axs[-1, 1].grid()
axs[-1, 1].legend([train_err, test_err], ["Training loss", "Testing loss"])   

plt.tight_layout()
plt.show()

# Plot the last training curve for all the different layers (to see the difference)

In [ ]:
fig, axs = plt.subplots(1,2, figsize = [10,10])
alphas = np.flip(np.linspace(1,0.2,len(layers_files)))

for layer in range(len(layers_files)):
    try:
        file = layers_files[layer][-1]
        df_classifier = pd.read_csv("Classifiers/"+ file, sep = ',', index_col = 0)
        train_err, = axs[0].plot(df_classifier["Training error"].values, color = "firebrick", alpha = alphas[layer], label = "Training error")
        test_err, = axs[0].plot(df_classifier["Testing error"].values, color = "forestgreen", alpha = alphas[layer], label = "Testing error")
        axs[0].set_ylim(0, np.max(df_classifier["Training error"].values))

        train_loss, = axs[1].plot(df_classifier["Training loss"].values, "--", color = "firebrick", alpha = alphas[layer], label = "Training loss")
        test_loss, = axs[1].plot(df_classifier["Testing loss"].values, "--", color = "forestgreen", alpha = alphas[layer], label = "Testing loss")
        axs[1].set_ylim(0, np.max(df_classifier["Training loss"].values))
        
       
            
    except:
        pass
    
axs[0].grid()
axs[0].legend([train_err, test_err], ["Training error", "Testing error"])

axs[1].grid()
axs[1].legend([train_err, test_err], ["Training loss", "Testing loss"])
plt.show()

# Plot in an interactive way the weights of the first layer over epochs

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import Image, display, clear_output, IFrame
from pdf2image import convert_from_path
import io
import imageio

In [ ]:
# load data from the weights folder and sort by epochs
files_weights = os.listdir('Weights')
files_weights.sort(key=lambda x: int(x.split("#")[1][:-4]))

In [ ]:
# Convert PDF to images (one image per page)
png_images = []
for idx, file in enumerate(files_weights):
    png_images.append(convert_from_path("Weights/" + file)[0])

In [ ]:
def display_pdf_image(image_idx):
    clear_output(wait=False)
    display(png_images[image_idx][0])

In [ ]:
image_slider = widgets.IntSlider(
    min=0,
    max=len(files_weights) - 1,
    step=1,
    value=0,
    description='Image:',
    width='300px'
)

In [ ]:
interactive_display = widgets.interactive(display_pdf_image, image_idx=image_slider)

In [ ]:
output_widget = widgets.Output()

In [ ]:
display(image_slider, output_widget)

In [ ]:
# Define a function to update the displayed image
def update_image(change):
    with output_widget:
        clear_output(wait=True)  # Clear the previous output
        display(png_images[change.new].resize((600,600)))

In [ ]:
# Connect the slider to the update_image function
image_slider.observe(update_image, 'value')

# Display the initial image
update_image({'new': 0})

In [ ]:
imageio.mimsave('evolution_weights.gif', png_images)